**CLOUD MASKING**
---

**Preface**

Satellite imagery is very useful for monitoring what is happening on earth surface. It has solved various problems, including wildfire related work.

However, we don't always have good imagery to begin with, some days are particularly cloudy. Which makes the problem even harder, we simply cannot see through it.

    + Even if some Machine Learning model predicts what is under the cloud, it is just pure inference, and I believe it has no value at all.

But, if it is still possible to enhance the details of data under "thin enough" enough, it can be useful in lots of ways.

On the notebook. "cloud reduction", I have created a model to work on that idea. It is not perfect, one problem is the given cloud masking is not good enough.

    -> In this notebook, we will go over the analysis which is helpful for cloud masking. 

**Change dir**

In [2]:
import os

os.chdir('/home/bill/GitHub/wps-research/data/bill')

**Prepare Data**

In [8]:
from raster import Raster
import numpy as np

In [ ]:
img_f = 'C11659/L1C/S2B_MSIL1C_20250902T192909_N0511_R142_T09UYU_20250902T230923.bin'

In [6]:
img = Raster(img_f)

for i, b in enumerate(img.band_info_list):
    print(f'{i + 1}: {b}')

1: 20250902 60m: B1 443nm
2: 20250902 10m: B2 490nm
3: 20250902 10m: B3 560nm
4: 20250902 10m: B4 665nm
5: 20250902 20m: B5 705nm
6: 20250902 20m: B6 740nm
7: 20250902 20m: B7 783nm
8: 20250902 10m: B8 842nm
9: 20250902 20m: B8A 865nm
10: 20250902 60m: B9 945nm
11: 20250902 20m: B11 1610nm
12: 20250902 20m: B12 2190nm


In [19]:
img_dat = img.read_bands([1,2,3,4,5,8,9,10,11,12], info=True)

---------------------
Your selected bands:
20250902 60m: B1 443nm
20250902 10m: B2 490nm
20250902 10m: B3 560nm
20250902 10m: B4 665nm
20250902 20m: B5 705nm
20250902 10m: B8 842nm
20250902 20m: B8A 865nm
20250902 60m: B9 945nm
20250902 20m: B11 1610nm
20250902 20m: B12 2190nm
---------------------
Data Dimension: (5490, 5490, 10)


**S2Cloudless**

In [1]:
from s2cloudless import S2PixelCloudDetector

In [20]:
if img_dat.max() > 1.0:
    img_dat /= 10000.0

In [12]:
detector = S2PixelCloudDetector(
    threshold=0.4,         # used only if you call get_cloud_masks
    average_over=4,        # spatial smoothing window
    dilation_size=2,       # grow clouds slightly
    all_bands=True
)

In [21]:
img_dat_ = np.expand_dims(img_dat, axis=0)

In [22]:
img_dat_.shape

(1, 5490, 5490, 10)

In [23]:
cloud_prob = detector.get_cloud_probability_maps(img_dat_)

ValueError: Parameter `all_bands` is set to True, but images have 10 bands.